# SnapPoint 신경망

In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import utils

from PIL import Image
import os
import PIL.Image as pilimg
import numpy as np
import pandas as pd
from keras.preprocessing import image

import matplotlib.pyplot as plt

In [160]:
data_x = np.load('np/data.npy')
data_y = np.load('np/data_label.npy')

In [161]:
data_x.shape

(2227, 5, 40)

In [162]:
n=int(2227*0.85)
n

1892

In [163]:
data_x[n:].shape

(335, 5, 40)

In [164]:
train_x = data_x[:n]
train_y = data_y[:n]
test_x = data_x[n:]
test_y = data_y[n:]

In [165]:
from keras.utils import to_categorical
# Step 1. Data Preprocessing
train_x=np.reshape(train_x,(-1,train_x.shape[1],train_x.shape[2],1))
test_x=np.reshape(test_x,(-1,test_x.shape[1],test_x.shape[2],1))
train_y = utils.to_categorical(train_y,10)
test_y = utils.to_categorical(test_y,10)

In [166]:
train_x.shape
train_y.shape

(1892, 10)

In [254]:
pool_size = (2, 2)  # size of pooling area for max pooling
kernel_size = (3, 3)  # convolution kernel size
np.random.seed(1337)

model = tf.keras.models.Sequential()
model.add(Conv2D(8, kernel_size, padding='same', input_shape=(5,40,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))

model.add(Conv2D(16, kernel_size, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(600))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile (optimizer= 'adam', 
                loss='binary_crossentropy', metrics = ['accuracy'])

In [255]:
model.fit(train_x, train_y, batch_size=128, epochs=12, verbose=2, validation_split=0.2)

Train on 1513 samples, validate on 379 samples
Epoch 1/12
1513/1513 - 0s - loss: 0.9996 - accuracy: 0.9141 - val_loss: 3.0438 - val_accuracy: 0.8000
Epoch 2/12
1513/1513 - 0s - loss: 0.5917 - accuracy: 0.9433 - val_loss: 2.1209 - val_accuracy: 0.8084
Epoch 3/12
1513/1513 - 0s - loss: 0.4094 - accuracy: 0.9581 - val_loss: 1.0072 - val_accuracy: 0.8712
Epoch 4/12
1513/1513 - 0s - loss: 0.2875 - accuracy: 0.9634 - val_loss: 0.6112 - val_accuracy: 0.9061
Epoch 5/12
1513/1513 - 0s - loss: 0.1843 - accuracy: 0.9722 - val_loss: 0.2772 - val_accuracy: 0.9425
Epoch 6/12
1513/1513 - 0s - loss: 0.1505 - accuracy: 0.9762 - val_loss: 0.2193 - val_accuracy: 0.9525
Epoch 7/12
1513/1513 - 0s - loss: 0.1172 - accuracy: 0.9779 - val_loss: 0.2689 - val_accuracy: 0.9435
Epoch 8/12
1513/1513 - 0s - loss: 0.0898 - accuracy: 0.9800 - val_loss: 0.1621 - val_accuracy: 0.9625
Epoch 9/12
1513/1513 - 0s - loss: 0.0758 - accuracy: 0.9822 - val_loss: 0.2034 - val_accuracy: 0.9504
Epoch 10/12
1513/1513 - 0s - loss: 

In [256]:
test_loss, test_acc = model.evaluate(test_x, test_y, verbose=2)
print('test_loss = ', test_loss, 'test_acc = ', test_acc)

predictions = model.predict(test_x)

335/1 - 0s - loss: 0.1276 - accuracy: 0.9534
test_loss =  0.19244698857638373 test_acc =  0.9534328


In [257]:
model.save('SnapPoint.h5')

In [264]:
import glob
import librosa
def cutting_sound(filepath):
    batch_size = 0
    data_height = 5
    data_width = 40
    train = np.zeros((batch_size, data_height, data_width))

    file_list = glob.glob(filepath + '*.wav')
    for file_name in file_list:
        y, sr = librosa.load(file_name, sr=20000)
        y_mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40, hop_length=500)
        _, beat_frames = librosa.beat.beat_track(y=y, sr=sr, hop_length=500)
        for frame_number in beat_frames:
            listSound = np.expand_dims(y_mfcc.T[frame_number-1:frame_number+4],axis=0)
            train = np.concatenate((train,listSound),axis=0)
    return train

In [290]:
sample=cutting_sound("./test/")
sample=np.reshape(sample,(-1,sample.shape[1],sample.shape[2],1))
predictions = model.predict(sample)
sample_y = utils.to_categorical(np.ones(8),10)
sample_y

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [291]:
cnt=0
for i in predictions:
    if np.argmax(i) == 1:
        print("great ", cnt+1)
    else:
        print("failed")
    cnt+=1

great  1
great  2
great  3
great  4
great  5
great  6
great  7
great  8


In [288]:
predictions

array([[1.67478658e-02, 9.82653916e-01, 8.57273044e-05, 5.33437851e-05,
        8.33375889e-05, 3.09740535e-05, 5.13193008e-05, 2.78371572e-05,
        6.94553773e-06, 2.58727203e-04],
       [3.99518646e-02, 9.53156352e-01, 6.98683027e-04, 9.01997846e-04,
        1.22854800e-03, 9.12750431e-04, 5.59262873e-04, 7.77408713e-04,
        1.52117093e-04, 1.66110659e-03],
       [1.26863718e-01, 8.54404211e-01, 1.46091043e-03, 3.98336444e-03,
        3.80505878e-03, 8.16942484e-04, 2.38740584e-03, 1.78354804e-03,
        3.59150261e-04, 4.13575396e-03],
       [7.32165277e-02, 9.18803096e-01, 9.09890921e-04, 1.57846848e-03,
        1.18899497e-03, 6.57666998e-04, 1.00478681e-03, 8.83436122e-04,
        2.54236511e-04, 1.50281773e-03],
       [1.50058803e-03, 9.98321116e-01, 3.01190230e-05, 3.73838084e-05,
        3.61244638e-05, 5.80593951e-06, 2.88122501e-05, 1.09512284e-05,
        1.53256929e-06, 2.75999701e-05],
       [2.63508886e-01, 5.65124154e-01, 1.76612996e-02, 2.87296921e-02,
   